## Installing Explainer Dashboard in SageMaker Studio

[Explainer Dashboard](https://explainerdashboard.readthedocs.io/en/latest/#) is a Python library that data scientists use for quickly building interactive dashboards for analyzing and explaining the predictions and workings of (scikit-learn compatible) machine learning models, including xgboost, catboost and lightgbm.

In this guide we will try to install and use Explainer Dashboard inside SageMaker Studio.

The notebook has been tested on the `Python 3(Data Science)` notebook.

### Install the Explainer Dashboard library

1. Install the dashboard

In [ ]:
%pip install scikit-learn --upgrade

In [ ]:
%pip install git+https://github.com/absynthe/explainerdashboard.git@master -q

2. Downgrade the Jinja2 library, otherwise the package will throw errors. 

In [ ]:
%pip install Jinja2==3.0.3 -q

### Visualize an Explainer Dashboard 

Prepare some data for demonstration purposes. To keep things simple, we will import the [Diabetes Dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset) from Scikit Learn. 

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from explainerdashboard import ClassifierExplainer, RegressionExplainer

In [ ]:
#Import the Diabetes Dataset
from sklearn.datasets import load_diabetes
data= load_diabetes()
print(data)

In [ ]:
#create a DataFrame from the dataset
X=pd.DataFrame(data.data,columns=data.feature_names)
X.head()

In [ ]:
y=pd.DataFrame(data.target,columns=["target"])
y.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from explainerdashboard import RegressionExplainer, ExplainerDashboard
import dash_bootstrap_components as dbc
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
y_train

In [ ]:
model = RandomForestRegressor(n_estimators=50, max_depth=5)
model.fit(X_train, y_train.values.ravel())


The default dash server is started with mode=`dash`. 

There are three notebook compatible options: 
- `mode='inline'` for running the dashboard in an output cell in your notebook
- `mode='jupyterlab'` for runnning the dashboard in jupyterlab pane
- `mode='external'` which runs the dashboard in a seperate tab

We will try all the suggested methods below. 

### Tying it all together


We are now finally ready to export the dashboard to EFS.

In [ ]:
from flask import Flask

explainer = RegressionExplainer(model, X_test, y_test)

PORT = 8051

db = ExplainerDashboard(explainer, 
                        title="Diabetes Prediction", 
                        whatif=False,
                        mode='dash',
                        routes_pathname_prefix='/', 
                        requests_pathname_prefix=f'/jupyter/default/proxy/{PORT}/'
                        )

print(db.app.config)

db.to_yaml("dashboard.yaml", explainerfile="dashboard.joblib", dump_explainer=True)

- Open the Jupyer App Terminal
- conda activate studio
- Install explainerdashboard again
- Navigate to the folder where the dashboard yaml is stored
- Now run the following command:
explainerdashboard run dashboard.yaml --port 8051

Dashboard is available at: < STUDIO_URL >/jupyter/default/proxy/< PORT >